# Imports

In [2]:
! nvidia-smi

Tue Oct 13 03:51:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [47]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, ReLU, ELU, LeakyReLU, Flatten, Dense, AveragePooling2D

import numpy as np
np.random.seed(666)
tf.random.set_seed(666)

In [59]:
options = {
    'a_filter_size': [(3,3), (5,5), (7,7), (13,13)],
    'a_include_BN': [True, False],
    'a_output_channels': [32, 64, 128, 256],
    'activation_type': [ReLU, ELU, LeakyReLU],
    'b_filter_size': [(3,3), (5,5), (7,7), (13,13)],
    'b_include_BN': [True, False],
    'b_output_channels': [32, 64, 128, 256],
    'include_pool': [True, False],
    'pool_type': [MaxPool2D, AveragePooling2D]
    }

In [6]:
# Load the training and testing set of CIFAR10
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

X_train = X_train.astype('float32')
X_train = X_train/255.

X_test = X_test.astype('float32')
X_test = X_test/255.

y_train = tf.reshape(tf.one_hot(y_train, 10), shape=(-1, 10))
y_test = tf.reshape(tf.one_hot(y_test, 10), shape=(-1, 10))

170500096/170498071 [==============================] - 4s 0us/step


In [7]:
# Create TensorFlow dataset
BATCH_SIZE = 256
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(1024).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds = test_ds.cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [8]:
for image, label in train_ds.take(1):
  print(image.shape)
  print(label.shape)

(256, 32, 32, 3)
(256, 10)


In [65]:
class Organism:
    def __init__(self, chromosome={}, phase=0):
      self.phase = phase
      self.chromosome = chromosome
    
    def build_model(self):
      if self.phase == 0:
        keras.backend.clear_session()
        
        inputs = Input(shape=(32,32,3))
        x = Conv2D(filters=self.chromosome['a_output_channels'],
                    kernel_size=self.chromosome['a_filter_size'],
                    use_bias=self.chromosome['a_include_BN'])(inputs)
        if self.chromosome['a_include_BN']:
            x = BatchNormalization()(x)
        x = self.chromosome['activation_type']()(x)
        if self.chromosome['include_pool']:
            x = self.chromosome['pool_type']()(x)

        x = Conv2D(filters=self.chromosome['b_output_channels'],
                    kernel_size=self.chromosome['b_filter_size'],
                    use_bias=self.chromosome['b_include_BN'])(x)
        if self.chromosome['b_include_BN']:
            x = BatchNormalization()(x)
        x = self.chromosome['activation_type']()(x)

        x = Flatten()(x)
        x = Dense(10, activation='softmax')(x)

        self.model = tf.keras.Model(inputs=[inputs], outputs=[x])
        self.model.compile(optimizer='adam',
                            loss='categorical_crossentropy',
                            metrics=['accuracy'])
      else:
        print('Phase under construction')
    
    def fitnessFunction(self, train_ds, test_ds):
      self.model.fit(train_ds,
                      epochs=5,
                      verbose=1)
      _, self.fitness = self.model.evaluate(test_ds, verbose=0)
    
    def crossover(self, partner):
      child = Organism(chromosome={}, phase=0)
      endpoint = np.random.randint(low=0, high=len(self.chromosome))
      print(endpoint)
      for idx, key in enumerate(self.chromosome):
        if idx <= endpoint:
          child.chromosome[key] = self.chromosome[key]
        else:
          child.chromosome[key] = partner.chromosome[key]
      return child
    
    def mutation(self, mutationRate):
      for idx, key in enumerate(self.chromosome):
        if np.random.rand() <= mutationRate:
          self.chromosome[key] = options[key][np.random.randint(len(options[key]))]

In [60]:
org1 = Organism({
    'a_filter_size': options['a_filter_size'][np.random.randint(len(options['a_filter_size']))],
    'a_include_BN': options['a_include_BN'][np.random.randint(len(options['a_include_BN']))],
    'a_output_channels': options['a_output_channels'][np.random.randint(len(options['a_output_channels']))],
    'activation_type': options['activation_type'][np.random.randint(len(options['activation_type']))],
    'b_filter_size': options['b_filter_size'][np.random.randint(len(options['b_filter_size']))],
    'b_include_BN': options['b_include_BN'][np.random.randint(len(options['b_include_BN']))],
    'b_output_channels': options['b_output_channels'][np.random.randint(len(options['b_output_channels']))],
    'include_pool': options['include_pool'][np.random.randint(len(options['include_pool']))],
    'pool_type': options['pool_type'][np.random.randint(len(options['pool_type']))]
    },
    phase=0)

org2 = Organism({
    'a_filter_size': options['a_filter_size'][np.random.randint(len(options['a_filter_size']))],
    'a_include_BN': options['a_include_BN'][np.random.randint(len(options['a_include_BN']))],
    'a_output_channels': options['a_output_channels'][np.random.randint(len(options['a_output_channels']))],
    'activation_type': options['activation_type'][np.random.randint(len(options['activation_type']))],
    'b_filter_size': options['b_filter_size'][np.random.randint(len(options['b_filter_size']))],
    'b_include_BN': options['b_include_BN'][np.random.randint(len(options['b_include_BN']))],
    'b_output_channels': options['b_output_channels'][np.random.randint(len(options['b_output_channels']))],
    'include_pool': options['include_pool'][np.random.randint(len(options['include_pool']))],
    'pool_type': options['pool_type'][np.random.randint(len(options['pool_type']))]
    },
    phase=0)

org1.build_model()
org2.build_model()

In [61]:
org1.fitnessFunction(train_ds, test_ds)
org2.fitnessFunction(train_ds, test_ds)

Epoch 1/5
196/196 [==============================] - 3s 14ms/step - loss: 1.9062 - accuracy: 0.3062
Epoch 2/5
196/196 [==============================] - 3s 13ms/step - loss: 1.6395 - accuracy: 0.4103
Epoch 3/5
196/196 [==============================] - 3s 13ms/step - loss: 1.5513 - accuracy: 0.4434
Epoch 4/5
196/196 [==============================] - 3s 13ms/step - loss: 1.5005 - accuracy: 0.4649
Epoch 5/5
196/196 [==============================] - 3s 13ms/step - loss: 1.4618 - accuracy: 0.4798
Epoch 1/5
196/196 [==============================] - 1s 6ms/step - loss: 1.6048 - accuracy: 0.4201
Epoch 2/5
196/196 [==============================] - 1s 6ms/step - loss: 1.3228 - accuracy: 0.5271
Epoch 3/5
196/196 [==============================] - 1s 6ms/step - loss: 1.2032 - accuracy: 0.5728
Epoch 4/5
196/196 [==============================] - 1s 6ms/step - loss: 1.1258 - accuracy: 0.6023
Epoch 5/5
196/196 [==============================] - 1s 6ms/step - loss: 1.0683 - accuracy: 0.6228


In [62]:
print('org1: {:0.2f}'.format(org1.fitness))
print('org2: {:0.2f}'.format(org2.fitness))

org1: 0.47
org2: 0.51


In [71]:
org1_2 = org1.crossover(org2)
org1_2.mutation(0.1)
org1_2.chromosome

4


{'a_filter_size': (5, 5),
 'a_include_BN': False,
 'a_output_channels': 128,
 'activation_type': tensorflow.python.keras.layers.advanced_activations.ReLU,
 'b_filter_size': (3, 3),
 'b_include_BN': False,
 'b_output_channels': 32,
 'include_pool': True,
 'pool_type': tensorflow.python.keras.layers.pooling.MaxPooling2D}

In [72]:
org1_2.build_model()
org1_2.fitnessFunction(train_ds, test_ds)

Epoch 1/5
196/196 [==============================] - 2s 9ms/step - loss: 1.6899 - accuracy: 0.3977
Epoch 2/5
196/196 [==============================] - 2s 9ms/step - loss: 1.3858 - accuracy: 0.5107
Epoch 3/5
196/196 [==============================] - 2s 9ms/step - loss: 1.2753 - accuracy: 0.5540
Epoch 4/5
196/196 [==============================] - 2s 9ms/step - loss: 1.1914 - accuracy: 0.5869
Epoch 5/5
196/196 [==============================] - 2s 9ms/step - loss: 1.1076 - accuracy: 0.6175


In [73]:
print('org1_2: {:0.2f}'.format(org1_2.fitness))

org1_2: 0.59
